# Solutions for pre-class assignment \#6

For the matrices A and B in the included
file `matrix_1.py`, write two functions that use Gaussian
elimination with partial pivoting to:

1. get the upper-triangular matrix, and
2. calculate the determinant of the matrix using the upper-triangular matrix.  

Do this without destroying the original
matrix by making a copy of the matrix (with `numpy.copy`) and
compare your calculated determinant to the output of the [`scipy.linalg.det`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.linalg.det.html#scipy.linalg.det) routine, as runon the same matrix.  Suggestion: invent your own
  small matrix (perhaps a $3 \times 3$ matrix) to test your routines on so that
  you can verify that it's behaving correctly on a step-by-step basis!


In [ ]:
#### FROM matrix_1.py, copied for convenience
import numpy as np

# simple 5x5 array - can calulate this pretty easily.
A = np.array( [[1,-3,2,-1,-2],[-2,2,-1,2,3],[3,-3,-2,1,-1],[1,-2,1,-3,2],[-3,-1,2,1,-3]],dtype='float64')

np.random.seed(8675309)

# much uglier, but should work the same.
B = np.random.rand(6,6)

# note: you can get the shape of an array A (and thus its dimensions) with A.shape(),
# and can print out its first (second) dimension with A.shape[0]  (A.shape[1])


In [ ]:
# this makes things a bit prettier
np.set_printoptions(precision=3)

#We're going to need this too
import scipy.linalg

In [ ]:
def partial_pivot(array,pivot_row,col):
    '''
    Example partial pivoting code.  Given an array, a pivot row, and a 
    column about which to pivot, figure out which [row,column] below that
    has the largest value (if any) and swap rows with that.
    '''
    swap_row = pivot_row
    for i in range(pivot_row,array.shape[0]):
        if np.abs(array[i,col]) > np.abs(array[pivot_row,col]):
            swap_row = i
    
    # if there IS a row with a large value, swap.
    if swap_row > pivot_row:
        row1 = np.copy(array[pivot_row,])
        row2 = np.copy(array[swap_row,])

        array[pivot_row,] = row2
        array[swap_row,] = row1

def gauss_elim(array):
    ''' 
    Gaussian elimination code using partial-pivoting.
    Step through rows of a given NxN array.  In each row i,
    do a partial pivot with rows below that to ensure that this row
    has the largest coefficient in column i, and then use Gaussian
    elimination in rows below that to zero out that column.  The
    resulting array will be an upper-triangle array, and replaces the 
    array that was fed into this function.  Also, we keep track of the
    order of the original rows for use in calculating the sign of the 
    determinant.
    
    NOTE: for the sake of clarity, I am NOT keeping track of the 'b' coefficients!
    '''
    
    if array.shape[0] != array.shape[1]:
        print("careful!  assumption broken!")

    # array used to keep track of original ordering
    ordering = np.arange(array.shape[0])
    
    # loop over each row
    for i in range(array.shape[0]):

        '''
        Chunk of code below this does partial-pivoting - swaps row i with a
        higher row if the value in column i is bigger.
        '''
        pivot_row = swap_row = i

        # check to see if we want to swap
        for k in range(pivot_row,array.shape[0]):
            if np.abs(array[k,i]) > np.abs(array[pivot_row,i]):
                swap_row = k

        # here's where we do the swapping
        if swap_row > pivot_row:
                        
            row1 = np.copy(array[pivot_row,])
            row2 = np.copy(array[swap_row,])

            array[pivot_row,] = row2
            array[swap_row,] = row1
            
            # in our ordering array, also swap the row ordering
            tmp = ordering[i] 
            ordering[i] = ordering[swap_row] 
            ordering[swap_row] = tmp

        # this is where we do the actual Gaussian elimination of all rows > i to
        # get rid of values in this column (zero them out)
        for j in range(i+1,array.shape[0]):

            test_row = np.copy(array[i,])
            test_row *= -float(array[j,i]/array[i,i])
            array[j,] += test_row

    return ordering


In [ ]:
def determinant(array):
    '''
    Calculate determinant of an array using Gaussian elimination.
    When we do Gaussian elimination, we get an upper-triangular array,
    and the determinant is just the product of the diagonal coefficients, modulo
    a sign flip.  Since we kept track of the sign flips, we can undo that.
    '''
    
    # get upper-triangular matrix, keeping track of sign flips
    indices = gauss_elim(array)

    # calculate determinant (with possibly wrong sign)
    determinant = 1
    for i in range(array.shape[0]):
        determinant *= array[i,i]

    '''
    Now we calculate the sign.  There is one sign flip per pair of swapped
    rows in our Gaussian elimination routine.  We kept track of where each row started
    using the array 'indices', so as we go through this array we can 'unwind' the swapping
    and flip the sign of the determinant every time we have to swap.
    '''
    
    for i in range(indices.size):

        # if i != indices, we have swapped this row.  Swap sign and swap vals in index so
        # we don't double-count.
        if i != indices[i]:
            determinant = -determinant
            j = indices[i]
            indices[i] = indices[j]
            indices[j] = j

    return determinant


## now let's do some testing.

In [ ]:
# WITH ARRAY A

# copy original array
newA = np.copy(A)

print("original array:\n",newA)

gauss_elim(newA)

print("\nafter Gaussian elimination:\n",newA)

# copy it again
newA = np.copy(A)
my_det = determinant(newA)

scipy_det = scipy.linalg.det(A)

print("\nmy value of the determinant:", my_det, "\tScipy's:", scipy_det)

In [ ]:
# WITH ARRAY B

# copy original array
newB = np.copy(B)

print("original array:\n",newB)

gauss_elim(newB)

print("\nafter Gaussian elimination:\n",newB)

# copy it again
newB = np.copy(B)
my_det = determinant(newB)

scipy_det = scipy.linalg.det(B)

print("\nmy value of the determinant:", my_det, "\tScipy's:", scipy_det)
